<a href="https://colab.research.google.com/github/hjn14133/DSP/blob/main/SignalTransmission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analog vs Digital Transmission
In this notebook we will explore the potential advantages of digital transmission over analog transmission. We will consider the case of transmission over a long (e.g. transoceanic) cable in which several repeaters are used to compensate for the attenuation introduced by the transmission.

Remember that if each cable segment introduces an attenuation of  1/G , we can recover the original amplitude by boosting the signal with a repeater with gain  GG . However, if the signal has accumulated additive noise, the noise will be amplified as well so that, after  N  repeaters, the noise will have been amplified  N  times:

x̂ N(t)=x(t)+NGσ(t)
 
If we use a digital signal, on the other hand, we can threshold the signal after each repeater and virtually eliminate the noise at each stage, so that even after several repeaters the trasmission is still noise-free.

In [3]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import IPython
from scipy.io import wavfile

In [5]:
plt.rcParams["figure.figsize"] = (14,4)


In [ ]:
rate, s = wavfile.read('speech.wav')
plt.plot(s);
IPython.display.Audio(s, rate=rate)

In [ ]:
# The Analog and Digital Signals
# analog audio signals - simulation - floating point values - [-100, 100] 

norm = 1.0 / max(np.absolute([min(s), max(s)]))

sA = 100.0 * s * norm

# digital version
sD = np.round(sA)

In [ ]:
plt.plot(sA-sD);

In [ ]:
# we will be computing SNRs later as well, so let's define a function
def SNR(noisy, original):
    # power of the error
    err = np.linalg.norm(original-noisy)
    # power of the signal
    sig = np.linalg.norm(original)
    # SNR in dBs
    return 10 * np.log10(sig/err)
    
    
print ('SNR = %f dB' % SNR(sD, sA))

In [ ]:
IPython.display.Audio(sA, rate=rate)

In [ ]:
IPython.display.Audio(sD, rate=rate)

In [ ]:
# Transmission with attenuation and noise - repeater 
def repeater(x, noise_amplitude, attenuation):
    # first, create the noise
    noise = np.random.uniform(-noise_amplitude, noise_amplitude, len(x))
    # attenuation
    x = x * attenuation
    # noise
    x = x + noise
    # gain compensation
    return x / attenuation

In [ ]:
# repeater for analog signals
def analog_tx(x, num_repeaters, noise_amplitude, attenuation):
    for n in range(0, num_repeaters):
        x = repeater(x, noise_amplitude, attenuation)
    return x

In [ ]:
# repeater for digital signals
def digital_tx(x, num_repeaters, noise_amplitude, attenuation):
    for n in range(0, num_repeaters):
        x = np.round(repeater(x, noise_amplitude, attenuation))
    return x

In [ ]:
# Compare tranmission schemes
NUM_REPEATERS = 70
NOISE_AMPLITUDE = 0.2
ATTENUATION = 0.5

yA = analog_tx(sA, NUM_REPEATERS, NOISE_AMPLITUDE, ATTENUATION)
print ('Analog trasmission: SNR = %f dB' % SNR(yA, sA))    

yD = digital_tx(sD, NUM_REPEATERS, NOISE_AMPLITUDE, ATTENUATION)
print ('Digital trasmission: SNR = %f dB' % SNR(yD, sA))    

In [ ]:
IPython.display.Audio(yA, rate=rate)

In [ ]:
IPython.display.Audio(yD, rate=rate)

In [ ]:
# if noise amp exceeds a certain values, digital transmission can be worse than analog one
NOISE_AMPLITUDE = 0.3

yA = analog_tx(sA, NUM_REPEATERS, NOISE_AMPLITUDE, ATTENUATION)
print ('Analog trasmission: SNR = %f dB' % SNR(yA, sA))   

yD = digital_tx(sD, NUM_REPEATERS, NOISE_AMPLITUDE, ATTENUATION)
print ('Digital trasmission: SNR = %f dB' % SNR(yD, sA))   